# Custom Fields Examples

This notebook demonstrates comprehensive custom field operations with the Neon CRM SDK, including:

- Finding custom fields by name and ID
- Searching accounts by custom field values
- Including custom field values in search results
- Best practices for custom field operations
- Migration analysis for existing custom fields

## What Are Custom Fields?

Custom fields allow organizations to extend their CRM with additional data fields specific to their needs. Common examples include:
- Volunteer interests and skills
- Industry classifications for companies
- Membership levels and preferences
- Communication preferences
- Legacy system IDs

## Prerequisites

Before running this notebook, make sure you have:
1. Installed the Neon CRM SDK: `pip install neon-crm`
2. Set up your environment variables:
   - `NEON_ORG_ID`: Your organization ID
   - `NEON_API_KEY`: Your API key
3. Or update the client initialization below with your credentials

## Import Required Libraries

In [ ]:
import os
from typing import Dict, Optional

from neon_crm import NeonClient

## Initialize the Neon CRM Client

In [ ]:
# Initialize the client with environment variables or direct credentials
client = NeonClient(
    org_id=os.getenv("NEON_ORG_ID"),
    api_key=os.getenv("NEON_API_KEY"),
    environment="production",  # or "trial"
)

# Alternative: Set credentials directly (not recommended for production)
# client = NeonClient(
#     org_id="your_org_id_here",
#     api_key="your_api_key_here",
#     environment="production"
# )

print("🎯 Neon CRM client initialized for custom fields examples!")

## Example 1: List All Custom Fields

Start by exploring what custom fields are available in your organization.

In [ ]:
def list_all_custom_fields():
    """List all custom fields for accounts."""
    print("📋 Available Custom Fields for Accounts:")
    print("=" * 50)

    try:
        custom_fields = list(client.accounts.list_custom_fields())

        if not custom_fields:
            print("ℹ️  No custom fields found for accounts")
            print("   This might mean:")
            print("   • Your organization hasn't set up custom fields yet")
            print("   • You don't have permission to view custom fields")
            print("   • The fields are configured differently")
            return []

        # Show first 10 fields with details
        print(f"Found {len(custom_fields)} custom fields. Showing details:")
        print()

        for i, field in enumerate(custom_fields[:10], 1):
            field_id = field.get("id")
            field_name = field.get("name", "Unknown")
            field_type = field.get("fieldType", "Unknown")

            print(
                f"{i:2}. ID: {field_id:3} | Type: {field_type:15} | Name: '{field_name}'"
            )

        if len(custom_fields) > 10:
            print(f"\n... and {len(custom_fields) - 10} more fields")

        print(f"\n✅ Found {len(custom_fields)} total custom fields")
        return custom_fields

    except Exception as e:
        print(f"❌ Error listing custom fields: {e}")
        return []


# Run the example
custom_fields = list_all_custom_fields()

## Example 2: Find Custom Field by Name

When you know the name of a custom field, you can find it efficiently.

In [ ]:
def find_custom_field_by_name(field_name: str) -> Optional[Dict]:
    """Find a custom field by its name."""
    print(f"🔍 Looking for custom field: '{field_name}'")

    try:
        field = client.accounts.find_custom_field_by_name(field_name)

        if field:
            field_id = field.get("id")
            field_type = field.get("fieldType")
            options = field.get("options", [])

            print("✅ Found field:")
            print(f"   ID: {field_id}")
            print(f"   Type: {field_type}")
            print(f"   Name: '{field_name}'")

            if options:
                print(
                    f"   Options: {', '.join(options[:5])}{'...' if len(options) > 5 else ''}"
                )

            return field
        else:
            print(f"❌ Custom field '{field_name}' not found")
            print("   💡 Tips:")
            print("      • Check spelling and capitalization")
            print("      • Use the exact field name from the list above")
            print("      • Some fields might be case-sensitive")
            return None

    except Exception as e:
        print(f"❌ Error finding custom field: {e}")
        return None


# Example: Find a custom field by name
print("\n🔍 Find Custom Field by Name")
print("=" * 35)

# If we have custom fields, try to find the first one
if custom_fields:
    example_field_name = custom_fields[0].get("name", "Unknown")
    print(f"Searching for the first field: '{example_field_name}'")
    found_field = find_custom_field_by_name(example_field_name)
else:
    print("⚠️  No custom fields available to search for")
    found_field = None

## Example 3: Search Accounts by Custom Field Values

One of the most powerful features is searching for accounts based on their custom field values.

In [ ]:
def search_by_custom_field(field_id: int, field_name: str):
    """Search for accounts with non-blank values in a custom field."""
    print(f"🔍 Searching accounts with non-blank '{field_name}' (ID: {field_id})")

    try:
        search_request = {
            "searchFields": [
                {"field": field_id, "operator": "NOT_BLANK"}  # Use integer field ID
            ],
            "outputFields": [
                "Account ID",
                "First Name",
                "Last Name",
                "Email 1",
                int(field_id),  # Include custom field in output (integer ID)
            ],
            "pagination": {"currentPage": 0, "pageSize": 200},  # Limit results for demo
        }

        results = list(client.accounts.search(search_request))

        print(f"✅ Found {len(results)} accounts with non-blank values")

        if results:
            print("\nSample results:")
            # Show sample results
            for i, account in enumerate(results[:5], 1):
                account_id = account.get("Account ID")
                first_name = account.get("First Name", "")
                last_name = account.get("Last Name", "")
                email = account.get("Email 1", "No email")

                # Custom field values are returned with display name as key
                custom_value = account.get(field_name, "N/A")

                name = f"{first_name} {last_name}".strip()
                print(f"  {i}. Account {account_id}: {name}")
                print(f"     Email: {email}")
                print(f"     {field_name}: '{custom_value}'")
                print()

            if len(results) > 5:
                print(f"    ... and {len(results) - 5} more accounts")
        else:
            print("   No accounts found with non-blank values in this field")

        return results

    except Exception as e:
        print(f"❌ Error searching by custom field: {e}")
        return []


# Run the search example if we have a custom field
print("\n🎯 Search by Custom Field Values")
print("=" * 35)

if found_field:
    field_id = found_field.get("id")
    field_name = found_field.get("name")
    search_results = search_by_custom_field(field_id, field_name)
else:
    print("⚠️  No custom field available for search example")
    search_results = []

## Example 4: Search by Specific Custom Field Value

Sometimes you want to find accounts with specific values in custom fields.

In [ ]:
def search_by_custom_field_value(field_id: int, field_name: str, search_value: str):
    """Search for accounts with specific custom field values."""
    print(f"🔍 Searching accounts where '{field_name}' contains '{search_value}'")

    try:
        search_request = {
            "searchFields": [
                {"field": field_id, "operator": "CONTAIN", "value": search_value}
            ],
            "outputFields": [
                "Account ID",
                "First Name",
                "Last Name",
                "Email 1",
                int(field_id),  # Include the custom field value
            ],
            "pagination": {"currentPage": 0, "pageSize": 200},
        }

        results = list(client.accounts.search(search_request))

        print(f"✅ Found {len(results)} accounts matching '{search_value}'")

        if results:
            print("\nMatching accounts:")
            for i, account in enumerate(results, 1):
                account_id = account.get("Account ID")
                first_name = account.get("First Name", "")
                last_name = account.get("Last Name", "")
                email = account.get("Email 1", "No email")
                custom_value = account.get(field_name, "N/A")

                name = f"{first_name} {last_name}".strip()
                print(f"  {i}. Account {account_id}: {name}")
                print(f"     Email: {email}")
                print(f"     {field_name}: '{custom_value}'")
                print()
        else:
            print(f"   No accounts found with '{search_value}' in {field_name}")

        return results

    except Exception as e:
        print(f"❌ Error searching by custom field value: {e}")
        return []


# Example: Search for specific value
print("\n🎯 Search by Specific Value")
print("=" * 30)

if found_field:
    field_id = found_field.get("id")
    field_name = found_field.get("name")

    # Common search terms - you can modify these
    search_terms = ["volunteer", "member", "yes", "active", "interested"]

    print(f"Trying common search terms in '{field_name}':")
    for term in search_terms[:2]:  # Try first 2 terms
        print(f"\n🔎 Searching for '{term}':")
        results = search_by_custom_field_value(field_id, field_name, term)
        if results:
            break  # Stop after finding matches
else:
    print("⚠️  No custom field available for value search example")

## Example 5: Multiple Custom Fields Search

Search using multiple custom fields at once to find accounts with specific combinations of data.

In [ ]:
def search_multiple_custom_fields(field_mapping: Dict[str, int]):
    """Search by multiple custom fields and include all in output."""
    print("🔍 Searching by multiple custom fields")
    print(f"Fields: {list(field_mapping.keys())}")

    # Build search fields for all custom fields
    search_fields = []
    output_fields = ["Account ID", "First Name", "Last Name", "Email 1"]

    for _field_name, field_id in field_mapping.items():
        search_fields.append({"field": field_id, "operator": "NOT_BLANK"})
        output_fields.append(int(field_id))  # Add to output

    try:
        search_request = {
            "searchFields": search_fields,
            "outputFields": output_fields,
            "pagination": {"currentPage": 0, "pageSize": 200},
        }

        results = list(client.accounts.search(search_request))

        print(
            f"✅ Found {len(results)} accounts with all specified custom fields populated"
        )

        if results:
            print("\nAccounts with all fields populated:")
            for i, account in enumerate(results, 1):
                account_id = account.get("Account ID")
                first_name = account.get("First Name", "")
                last_name = account.get("Last Name", "")
                email = account.get("Email 1", "No email")

                name = f"{first_name} {last_name}".strip()
                print(f"  {i}. Account {account_id}: {name}")
                print(f"     Email: {email}")

                # Show all custom field values
                for field_name in field_mapping.keys():
                    custom_value = account.get(field_name, "N/A")
                    print(f"     {field_name}: '{custom_value}'")
                print()
        else:
            print("   No accounts found with all specified fields populated")

        return results

    except Exception as e:
        print(f"❌ Error in multi-field search: {e}")
        return []


# Example: Multiple fields search
print("\n🎯 Multiple Custom Fields Search")
print("=" * 35)

if len(custom_fields) >= 2:
    # Use first 2 custom fields for the example
    field_mapping = {
        custom_fields[0]["name"]: custom_fields[0]["id"],
        custom_fields[1]["name"]: custom_fields[1]["id"],
    }
    multi_results = search_multiple_custom_fields(field_mapping)
else:
    print("⚠️  Need at least 2 custom fields for multi-field search example")
    if custom_fields:
        print(f"   Only {len(custom_fields)} custom field(s) available")

## Example 6: Custom Field Migration Analysis

This example shows how to analyze custom fields for data migration or cleanup purposes.

In [ ]:
def custom_field_migration_example():
    """Example of analyzing custom fields for migration (like V-fields)."""
    print("📊 Custom Field Migration Analysis Example")
    print("=" * 50)

    # Example target fields - these might be from a legacy system
    # You can modify these to match your actual field names
    target_fields = [
        "V-Annual Fundraising Event Committee",
        "V-Canvassing",
        "V-Coffee Klatch Interest",
        "V-Communications Team - website, newsletter, blog, messaging, graphic design",
        "V-Data Entry",
    ]

    # If we don't have these specific fields, use the first few available fields
    if not any(field.get("name") in target_fields for field in custom_fields):
        print("ℹ️  Example V-fields not found. Using available fields for demo:")
        target_fields = [field.get("name") for field in custom_fields[:5]]
        if not target_fields:
            print("❌ No custom fields available for migration analysis")
            return

    results_summary = {}

    for field_name in target_fields:
        print(f"\n🔍 Analyzing: {field_name}")

        # Find the field
        field = client.accounts.find_custom_field_by_name(field_name)
        if not field:
            print("  ❌ Field not found")
            results_summary[field_name] = {"status": "not_found"}
            continue

        field_id = field["id"]
        field_type = field.get("fieldType", "Unknown")
        print(f"  ✅ Found - ID: {field_id}, Type: {field_type}")

        try:
            # Count accounts with non-blank values
            search_request = {
                "searchFields": [{"field": field_id, "operator": "NOT_BLANK"}],
                "outputFields": ["Account ID"],
                "pagination": {"currentPage": 0, "pageSize": 100},  # Get more to count
            }

            # Get results to count them
            results = list(client.accounts.search(search_request))
            count = len(results)

            print(f"  📊 {count} accounts have non-blank values")

            # Get a sample of values if any exist
            if count > 0 and results:
                sample_search = {
                    "searchFields": [{"field": field_id, "operator": "NOT_BLANK"}],
                    "outputFields": ["Account ID", field_id],
                    "pagination": {"currentPage": 0, "pageSize": 200},
                }
                sample_results = list(client.accounts.search(sample_search))

                if sample_results:
                    print("  📝 Sample values:")
                    for result in sample_results:
                        value = result.get(field_name, "N/A")
                        account_id = result.get("Account ID")
                        print(f"     Account {account_id}: '{value}'")

            results_summary[field_name] = {
                "status": "found",
                "field_id": field_id,
                "field_type": field_type,
                "count": count,
            }

        except Exception as e:
            print(f"  ❌ Error analyzing field: {e}")
            results_summary[field_name] = {
                "status": "error",
                "field_id": field_id,
                "error": str(e),
            }

    # Print summary
    print("\n📊 MIGRATION ANALYSIS SUMMARY")
    print("=" * 35)

    total_records = sum(
        data.get("count", 0)
        for data in results_summary.values()
        if data.get("status") == "found"
    )

    found_fields = 0
    fields_with_data = 0

    for field_name, data in results_summary.items():
        status = data.get("status")

        if status == "not_found":
            print(f"❌ {field_name}: Field not found")
        elif status == "error":
            print(f"⚠️  {field_name}: Error occurred")
        elif status == "found":
            found_fields += 1
            count = data.get("count", 0)
            if count == 0:
                print(f"⭕ {field_name}: No data (0 records)")
            else:
                fields_with_data += 1
                print(f"✅ {field_name}: {count} records")

    print("\n🎯 Analysis Results:")
    print(f"   Total fields analyzed: {len(target_fields)}")
    print(f"   Fields found: {found_fields}")
    print(f"   Fields with data: {fields_with_data}")
    print(f"   Total records across all fields: {total_records}")

    if fields_with_data > 0:
        print("\n💡 Migration Recommendations:")
        print(
            f"   • {fields_with_data} fields contain data and should be included in migration"
        )
        print("   • Consider data validation and cleanup for fields with many records")
        print("   • Test migration process with a subset of accounts first")


# Run the migration analysis
print("\n📊 Custom Field Migration Analysis")
print("=" * 40)
custom_field_migration_example()

## Example 7: Custom Field Data Types and Validation

Understanding the different types of custom fields and their characteristics.

In [28]:
def analyze_custom_field_types():
    """Analyze the types and characteristics of custom fields."""
    print("🔍 Custom Field Types Analysis")
    print("=" * 35)

    if not custom_fields:
        print("❌ No custom fields available for analysis")
        return

    # Group fields by type
    field_types = {}

    for field in custom_fields:
        field_type = field.get("dataType", "Unknown")
        if field_type not in field_types:
            field_types[field_type] = []
        field_types[field_type].append(field)

    print(f"Found {len(field_types)} different field types:")
    print()

    for field_type, fields in field_types.items():
        print(f"📋 {field_type} Fields ({len(fields)} total):")

        # Show first few examples
        for i, field in enumerate(fields[:3], 1):
            field_id = field.get("id")
            field_name = field.get("name", "Unknown")

            print(f"   {i}. ID {field_id}: '{field_name}'")

            # Show options if available (for dropdown/checkbox fields)
            options = field.get("options", [])
            if options:
                option_preview = ", ".join(options[:3])
                if len(options) > 3:
                    option_preview += f"... (+{len(options) - 3} more)"
                print(f"      Options: {option_preview}")

        if len(fields) > 3:
            print(f"   ... and {len(fields) - 3} more {field_type} fields")
        print()

    # Field type explanations
    print("💡 Field Type Guide:")
    field_type_info = {
        "text": "Free-form text input",
        "textarea": "Multi-line text input",
        "dropdown": "Single selection from predefined options",
        "checkbox": "Multiple selections from predefined options",
        "date": "Date values",
        "number": "Numeric values",
        "currency": "Monetary values",
        "boolean": "Yes/No or True/False values",
    }

    for ftype in field_types.keys():
        if ftype is None:
            print(f'ftype is None')
            continue
        description = field_type_info.get(ftype.lower(), "Custom field type")
        print(f"   • {ftype}: {description}")


# Run the field types analysis
print("\n🔍 Field Types Analysis")
print("=" * 25)
analyze_custom_field_types()


🔍 Field Types Analysis
🔍 Custom Field Types Analysis
Found 3 different field types:

📋 Text Fields (1 total):
   1. ID 111: 'DPW ID'

📋 None Fields (40 total):
   1. ID 144: 'If bringing, indicate the dish you will share.'
   2. ID 101: 'Occupation'
   3. ID 113: 'V-Annual Fundraising Event Committee'
   ... and 37 more None fields

📋 Date Fields (2 total):
   1. ID 77: 'V-Date Contacted'
   2. ID 154: 'V-Volunteer Form Submitted Date'

💡 Field Type Guide:
   • Text: Free-form text input
ftype is None
   • Date: Date values


## Main Demonstration Function

Run all custom field examples in sequence:

In [ ]:
def main():
    """Run all custom field examples."""
    print("🎯 Neon CRM SDK - Custom Fields Examples")
    print("=" * 60)

    try:
        # Check if we have any custom fields to work with
        if not custom_fields:
            print("❌ No custom fields found. This could mean:")
            print("   • Your organization hasn't set up custom fields yet")
            print("   • You don't have permission to view custom fields")
            print("   • There was an error connecting to the API")
            print("\n💡 To use these examples:")
            print("   1. Set up custom fields in your Neon CRM admin panel")
            print("   2. Ensure your API key has appropriate permissions")
            print("   3. Check your organization ID and API key")
            return

        print(f"\n✅ Found {len(custom_fields)} custom fields to work with")
        print("\nRunning comprehensive custom fields examples...")

        # All examples have already been run above in their individual cells
        # This is a summary

        print("\n🎉 Custom Fields Examples Summary:")
        print("=" * 40)
        print("✅ Listed all available custom fields")
        print("✅ Demonstrated field lookup by name")
        print("✅ Showed searching by custom field values")
        print("✅ Performed multi-field searches")
        print("✅ Analyzed field types and characteristics")
        print("✅ Provided migration analysis example")

        print("\n📚 Key Learnings:")
        print("   • Custom fields extend CRM functionality")
        print("   • Use field IDs (integers) in search requests")
        print("   • Field names are used as keys in search results")
        print("   • Multiple operators available: NOT_BLANK, CONTAIN, EQUAL, etc.")
        print("   • Different field types support different operations")

        print("\n🚀 Next Steps:")
        print("   • Try creating accounts with custom field data")
        print("   • Explore updating custom field values")
        print("   • Build reports based on custom field data")
        print("   • Set up automated workflows using custom fields")

    except Exception as e:
        print(f"❌ Error running examples: {e}")
        print("💡 Please check:")
        print("   • Your network connection")
        print("   • Your API credentials (NEON_ORG_ID and NEON_API_KEY)")
        print("   • Your API permissions")


# Run the main demonstration
print("\n🎯 Running Main Custom Fields Demo")
print("=" * 40)
main()

## Cleanup

Don't forget to close the client connection when you're done:

In [ ]:
# Close the client connection
client.close()
print("✅ Client connection closed.")

## Summary and Best Practices

### What You Learned

This notebook demonstrated comprehensive custom field operations:

1. **Field Discovery**: How to list and explore available custom fields
2. **Field Lookup**: Finding fields by name with error handling
3. **Basic Search**: Finding accounts with non-blank custom field values
4. **Value Search**: Searching for specific values in custom fields
5. **Multi-Field Search**: Using multiple custom fields in a single query
6. **Migration Analysis**: Analyzing field usage for data migration
7. **Field Types**: Understanding different custom field types

### Best Practices

#### Search Operations
- Use **field IDs (integers)** in search requests, not field names
- Field **names are used as keys** in search results
- Always handle the case where fields might not exist
- Use appropriate operators: `NOT_BLANK`, `CONTAIN`, `EQUAL`, etc.

#### Performance
- Use pagination for large result sets
- Limit output fields to only what you need
- Consider caching field metadata for frequently used fields

#### Error Handling
- Always check if custom fields exist before using them
- Handle API errors gracefully
- Provide meaningful error messages to users

#### Field Types
- **Text/Textarea**: Use `CONTAIN` for partial matches, `EQUAL` for exact matches
- **Dropdown/Checkbox**: Use `EQUAL` with exact option values
- **Date**: Use date-specific operators and proper formatting
- **Number/Currency**: Use numeric operators and appropriate data types

### Common Use Cases

- **Volunteer Management**: Track skills, interests, and availability
- **Donor Segmentation**: Categorize donors by giving capacity or interests
- **Event Management**: Track event preferences and participation history
- **Communication Preferences**: Manage how contacts want to be reached
- **Legacy System Integration**: Store IDs from other systems

### Next Steps

Now that you understand custom fields, explore:

- **Account Creation with Custom Fields**: Set custom field values when creating accounts
- **Bulk Updates**: Update custom fields for multiple accounts
- **Reporting**: Build reports based on custom field data
- **Advanced Search**: Combine custom fields with standard fields in complex queries

Custom fields are one of the most powerful features of the Neon CRM SDK - use them to make your CRM work exactly the way your organization needs! 🎯